<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [10]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    # tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

In [ ]:
def DE(pop_size, iterations, function, limits):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    # function calls = pop_size + iterations * pop_size
    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    # print(X)
    amount_of_layer_1 = int(round(X[0]))
    amount_of_layer_2 = int(round(X[1]))
    amount_of_layer_3 = int(round(X[2]))
    amount_of_one_dence = int(round(X[3]))
    amount_of_one_filters = int(round(X[4]))
    # print(np.int64(np.round(X)))
    # print(amount_of_layer_1, amount_of_layer_2, amount_of_layer_3, amount_of_one_dence, amount_of_one_filters)
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    # model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_accuracy'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[0, 4], [0, 4], [0, 4], [0, 4], [4, 32]]


In [12]:
best_f, best_d = DE(4, 4, func, func_limits)
best_f, best_d

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 180s 540ms/step - Top2: 0.3663 - accuracy: 0.1947 - loss: 2.2317 - val_Top2: 0.2458 - val_accuracy: 0.1234 - val_loss: 3.0848
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 171s 546ms/step - Top2: 0.5622 - accuracy: 0.3194 - loss: 1.8025 - val_Top2: 0.6317 - val_accuracy: 0.3990 - val_loss: 1.6317
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 165s 527ms/step - Top2: 0.6261 - accuracy: 0.3836 - loss: 1.6673 - val_Top2: 0.6015 - val_accuracy: 0.3771 - val_loss: 1.7254
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 162s 518ms/step - Top2: 0.6632 - accuracy: 0.4295 - loss: 1.5697 - val_Top2: 0.6000 - val_accuracy: 0.4014 - val_loss: 1.8224
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 163s 520ms/step - Top2: 0.7016 - accuracy: 0.4734 - loss: 1.4630 - val_Top2: 0.6333 - val_accuracy: 0.4065 - val_loss: 1.6626
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 162s 517ms/step - Top2: 0.7275 - accuracy: 0.5054 - loss: 1.3789 - val_Top2: 0.6728 - val_accuracy: 0.4765 - val_loss: 1.573

Processing:   0%|                                        | 0/4 [00:00<?, ?step/s]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 162ms/step - Top2: 0.2978 - accuracy: 0.1581 - loss: 2.3565 - val_Top2: 0.2491 - val_accuracy: 0.1294 - val_loss: 2.8420
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 158ms/step - Top2: 0.4980 - accuracy: 0.2704 - loss: 1.9161 - val_Top2: 0.4821 - val_accuracy: 0.2610 - val_loss: 2.1711
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 50s 158ms/step - Top2: 0.5689 - accuracy: 0.3211 - loss: 1.8026 - val_Top2: 0.5551 - val_accuracy: 0.3092 - val_loss: 2.0318
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 50s 158ms/step - Top2: 0.6116 - accuracy: 0.3618 - loss: 1.7151 - val_Top2: 0.6375 - val_accuracy: 0.3998 - val_loss: 1.6303
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 157ms/step - Top2: 0.6367 - accuracy: 0.3944 - loss: 1.6515 - val_Top2: 0.6049 - val_accuracy: 0.3884 - val_loss: 1.7557
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 158ms/step - Top2: 0.6555 - accuracy: 0.4148 - loss: 1.5957 - val_Top2: 0.6465 - val_accuracy: 0.4232 - val_loss: 1.5910
Epoc

Processing:  25%|██████████                              | 1/4 [4:29:21<13:28:03, 16161.14s/step]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - Top2: 0.3787 - accuracy: 0.2011 - loss: 2.3175 - val_Top2: 0.2905 - val_accuracy: 0.1672 - val_loss: 2.4900
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 288s 919ms/step - Top2: 0.5678 - accuracy: 0.3351 - loss: 1.7954 - val_Top2: 0.5313 - val_accuracy: 0.2938 - val_loss: 1.9253
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 267s 854ms/step - Top2: 0.6281 - accuracy: 0.3980 - loss: 1.6445 - val_Top2: 0.6219 - val_accuracy: 0.4153 - val_loss: 1.6306
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 263s 841ms/step - Top2: 0.6753 - accuracy: 0.4503 - loss: 1.5204 - val_Top2: 0.6536 - val_accuracy: 0.4407 - val_loss: 1.6230
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 259s 829ms/step - Top2: 0.7023 - accuracy: 0.4851 - loss: 1.4423 - val_Top2: 0.6178 - val_accuracy: 0.4079 - val_loss: 1.7799
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 275s 879ms/step - Top2: 0.7349 - accuracy: 0.5176 - loss: 1.3499 - val_Top2: 0.7175 - val_accuracy: 0.5236 - val_loss: 1.3524
E

Processing:  25%|██████████                              | 1/4 [7:13:35<21:40:46, 26015.33s/step]


KeyboardInterrupt: 

In [ ]:
best_f, best_d

NameError: name 'best_f' is not defined

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)